In [9]:
def readin(filename):
    with open(filename) as f:
        lines = f.readlines()

    #something to remove \n 
    str_lines = []
    for l in lines:
        if l[-1:]=="\n":
            str_lines.append(l[0:-1])
        else: 
            str_lines.append(l)

    #check all was fine
    print(lines[-2:])
    print(str_lines[-2:])
    return str_lines

elf_data = readin("Day15input.txt")

['Sensor at x=1590740, y=3586256: closest beacon is at x=1532002, y=3577287\n', 'Sensor at x=1033496, y=3762565: closest beacon is at x=1532002, y=3577287\n']
['Sensor at x=1590740, y=3586256: closest beacon is at x=1532002, y=3577287', 'Sensor at x=1033496, y=3762565: closest beacon is at x=1532002, y=3577287']


# --- Day 15: Beacon Exclusion Zone ---

You feel the ground rumble again as the distress signal leads you to a large network of subterranean tunnels. You don't have time to search them all, but you don't need to: your pack contains a set of deployable sensors that you imagine were originally built to locate lost Elves.

The sensors aren't very powerful, but that's okay; your handheld device indicates that you're close enough to the source of the distress signal to use them. You pull the emergency sensor system out of your pack, hit the big button on top, and the sensors zoom off down the tunnels.

Once a sensor finds a spot it thinks will give it a good reading, it attaches itself to a hard surface and begins monitoring for the nearest signal source beacon. Sensors and beacons always exist at integer coordinates. Each sensor knows its own position and can determine the position of a beacon precisely; however, sensors can only lock on to the one beacon closest to the sensor as measured by the Manhattan distance. (There is never a tie where two beacons are the same distance to a sensor.)

It doesn't take long for the sensors to report back their positions and closest beacons (your puzzle input). For example:


In [10]:
test_data = ["Sensor at x=2, y=18: closest beacon is at x=-2, y=15",
"Sensor at x=9, y=16: closest beacon is at x=10, y=16",
"Sensor at x=13, y=2: closest beacon is at x=15, y=3",
"Sensor at x=12, y=14: closest beacon is at x=10, y=16",
"Sensor at x=10, y=20: closest beacon is at x=10, y=16",
"Sensor at x=14, y=17: closest beacon is at x=10, y=16",
"Sensor at x=8, y=7: closest beacon is at x=2, y=10",
"Sensor at x=2, y=0: closest beacon is at x=2, y=10",
"Sensor at x=0, y=11: closest beacon is at x=2, y=10",
"Sensor at x=20, y=14: closest beacon is at x=25, y=17",
"Sensor at x=17, y=20: closest beacon is at x=21, y=22",
"Sensor at x=16, y=7: closest beacon is at x=15, y=3",
"Sensor at x=14, y=3: closest beacon is at x=15, y=3",
"Sensor at x=20, y=1: closest beacon is at x=15, y=3"]

In [11]:
def parse_data(data):
    returnData = []
    for d in data:
        pair = d.split(":")
        sensor = pair[0].split(",")
        d1 = eval(sensor[0].removeprefix("Sensor at x="))
        d2 = eval(sensor[1].removeprefix(" y="))
        beacon = pair[1].split(",")
        d3 = eval(beacon[0].removeprefix(" closest beacon is at x="))
        d4 = eval(beacon[1].removeprefix(" y="))
        manhattan = abs(d3-d1) + abs(d4-d2)
        upper = d2+manhattan
        lower = d2-manhattan
        returnData.append([d1, d2, d3, d4, manhattan, upper, lower])
        #print(d1, d2, d3, d4)
    return returnData

parsed_test_data = parse_data(test_data)
parsed_elf_data = parse_data(elf_data)

import sys


So, consider the sensor at 2,18; the closest beacon to it is at -2,15. For the sensor at 9,16, the closest beacon to it is at 10,16.

Drawing sensors as S and beacons as B, the above arrangement of sensors and beacons looks like this:

                1    1    2    2
        0    5    0    5    0    5
    0 ....S.......................
    1 ......................S.....
    2 ...............S............
    3 ................SB..........
    4 ............................
    5 ............................
    6 ............................
    7 ..........S.......S.........
    8 ............................
    9 ............................
    10 ....B.......................
    11 ..S.........................
    12 ............................
    13 ............................
    14 ..............S.......S.....
    15 B...........................
    16 ...........SB...............
    17 ................S..........B
    18 ....S.......................
    19 ............................
    20 ............S......S........
    21 ............................
    22 .......................B....


In [12]:
def get_grid_dim(pd):
    min_x = pd[0][0]
    max_x = pd[0][0]
    # min_y = pd[0][1]
    # max_y = pd[0][1]
    for j in pd:
        # print(i)
        # print(j)
        # print(j[0])
        # print(j[1])
        # min_y = min(j[1], j[3], min_y)
        # max_y = max(j[1], j[3], max_y)
        min_x = min(j[0], j[2], min_x)
        max_x = max(j[0], j[2], max_x)

    return (min_x, max_x)


This isn't necessarily a comprehensive map of all beacons in the area, though. Because each sensor only identifies its closest beacon, if a sensor detects a beacon, you know there are no other beacons that close or closer to that sensor. There could still be beacons that just happen to not be the closest beacon to any sensor. Consider the sensor at 8,7:

                1    1    2    2
        0    5    0    5    0    5
    -2 ..........#.................
    -1 .........###................
    0 ....S...#####...............
    1 .......#######........S.....
    2 ......#########S............
    3 .....###########SB..........
    4 ....#############...........
    5 ...###############..........
    6 ..#################.........
    7 .#########S#######S#........
    8 ..#################.........
    9 ...###############..........
    10 ....B############...........
    11 ..S..###########............
    12 ......#########.............
    13 .......#######..............
    14 ........#####.S.......S.....
    15 B........###................
    16 ..........#SB...............
    17 ................S..........B
    18 ....S.......................
    19 ............................
    20 ............S......S........
    21 ............................
    22 .......................B....

This sensor's closest beacon is at 2,10, and so you know there are no beacons that close or closer (in any positions marked #).


In [13]:
def setup_row_from_scratch(min_x, max_x, row_num, parsed_data):
    new_line = []
    for x in range(min_x, max_x+1):    
        new_line.append(".")
    s = len(parsed_data)
    s0 = 0
    print("Empty row set up, checking ", s, " sensors")
    for sensor in parsed_data:
        sensor_x = sensor[0]-min_x
        sensor_y = sensor[1]  # no need to normalise to 0 index
        beacon_x = sensor[2]-min_x
        beacon_y = sensor[3]
        s0 = s0+1
        print("Checking sensor ", s0, " of ", s)
        if sensor_y == row_num:
            new_line[sensor_x] = "S"
            print("sensor added at ", sensor_x, row_num)
        elif beacon_y == row_num:
            new_line[beacon_x] =  "B"
            print("beacon added at ", beacon_x, row_num)
    print("Finished, returning row")
    return new_line


def check_row_from_scratch(row, row_no, min_x, max_x, parsed_data):
    for sensor in parsed_data:
        # returnData.append([d1, d2, d3, d4, manhattan, upper, lower])
        if row_no in range(sensor[6], sensor[5]): #affected
            sensor_x = sensor[0]-min_x
            sensor_y = sensor[1]
            manhattan = sensor[4]
            beacon_x = sensor[2]-min_x
            beacon_y = sensor[3]
            
            diff = abs(sensor_y - row_no) 
            x_range = manhattan - diff 
            x_left = max((sensor_x - x_range), 0)
            x_right = min((sensor_x + x_range), (len(row)-1))

            print("Checking sensor at ", sensor_x, sensor_y, " over range ", x_left, x_right, " with manhattan ", manhattan, " with diff ", diff, " x_range ", x_range, " and min_x is ", min_x)
            for x in range(x_left, x_right+1):
                if row[x]==".":
                    row[x] = "#"
                    #print(x, "#")
                # else: 
                    #print(x, row[x])
    na = row.count("#")
    return na


None of the detected beacons seem to be producing the distress signal, so you'll need to work out where the distress beacon is by working out where it isn't. For now, keep things simple by counting the positions where a beacon cannot possibly be along just a single row.

So, suppose you have an arrangement of beacons and sensors like in the example above and, just in the row where y=10, you'd like to count the number of positions a beacon cannot possibly exist. The coverage from all sensors near that row looks like this:

                    1    1    2    2
        0    5    0    5    0    5
    9 ...#########################...
    10 ..####B######################..
    11 .###S#############.###########.


In [14]:
def explore_row(row_num, parsed_data):
    (min_x, max_x) = get_grid_dim(parsed_data)
    print("get grid_dim ", min_x, max_x, " now starting setup row ", row_num)
    row = setup_row_from_scratch(min_x, max_x, row_num, parsed_data)
    print("Row set up length ", len(row), " starting to check row ", row_num)
    return check_row_from_scratch(row, row_num, min_x, max_x, parsed_data)

print(explore_row(10, parsed_test_data))
# (grid, min_y) = set_up_grid(parsed_elf_data)
# get_no_beacons_in_row(grid, 2000000, min_y)

get grid_dim  -2 25  now starting setup row  10
Empty row set up, checking  14  sensors
Checking sensor  1  of  14
Checking sensor  2  of  14
Checking sensor  3  of  14
Checking sensor  4  of  14
Checking sensor  5  of  14
Checking sensor  6  of  14
Checking sensor  7  of  14
beacon added at  4 10
Checking sensor  8  of  14
beacon added at  4 10
Checking sensor  9  of  14
beacon added at  4 10
Checking sensor  10  of  14
Checking sensor  11  of  14
Checking sensor  12  of  14
Checking sensor  13  of  14
Checking sensor  14  of  14
Finished, returning row
Row set up length  28  starting to check row  10
Checking sensor at  14 14  over range  14 14  with manhattan  4  with diff  4  x_range  0  and min_x is  -2
Checking sensor at  10 7  over range  4 16  with manhattan  9  with diff  3  x_range  6  and min_x is  -2
Checking sensor at  2 11  over range  0 4  with manhattan  3  with diff  1  x_range  2  and min_x is  -2
Checking sensor at  22 14  over range  18 26  with manhattan  8  with d


In this example, in the row where y=10, there are 26 positions where a beacon cannot be present.

Consult the report from the sensors you just deployed. In the row where y=2000000, how many positions cannot contain a beacon?

In [15]:
print(explore_row(2000000, parsed_elf_data))

get grid_dim  13820 3999189  now starting setup row  2000000
Empty row set up, checking  36  sensors
Checking sensor  1  of  36
Checking sensor  2  of  36
Checking sensor  3  of  36
Checking sensor  4  of  36
Checking sensor  5  of  36
Checking sensor  6  of  36
Checking sensor  7  of  36
Checking sensor  8  of  36
Checking sensor  9  of  36
Checking sensor  10  of  36
Checking sensor  11  of  36
Checking sensor  12  of  36
Checking sensor  13  of  36
Checking sensor  14  of  36
beacon added at  2318520 2000000
Checking sensor  15  of  36
Checking sensor  16  of  36
Checking sensor  17  of  36
Checking sensor  18  of  36
Checking sensor  19  of  36
Checking sensor  20  of  36
beacon added at  2318520 2000000
Checking sensor  21  of  36
Checking sensor  22  of  36
beacon added at  2318520 2000000
Checking sensor  23  of  36
Checking sensor  24  of  36
beacon added at  2318520 2000000
Checking sensor  25  of  36
beacon added at  2318520 2000000
Checking sensor  26  of  36
Checking sensor

* answer 3985369 is too low* 